# Информационный поиск и парсинг

Регулярные выражения  
Фразовые запросы и ранжированный информационный поиск;  
Методы оценки качества поисковых машин.  
Синтаксис составляющих и синтаксис зависимостей;  
Контекстонезависимые грамматики;  
Вероятностный подход к парсингу;  
Лексикализованные вероятностные грамматики;  
Применение парсинга в различных задачах.

Регулярные выражения. 
Модуль re.